# Convert to standard Vega-lite JSON

In [ ]:
#NOTE: 5 columns = 190MB, so just save SWE and Snow depth to save space for now...

In [ ]:
import glob
import pandas as pd

In [ ]:
#os.chdir('timeseries')

In [ ]:
files = glob.glob('*.parquet')
files[-1]

In [ ]:
df = pd.read_parquet(f)
df = df[['SNWD','WTEQ']].reset_index()

In [ ]:
# for f in files:
#     df = pd.read_parquet(f)
#     df = df[['SNWD','WTEQ']].reset_index()
#     df.to_json('./json/'+f.replace('.parquet','.json'), 
#              orient='records', 
#              #date_unit='s',
#              date_format='iso',
#              double_precision=5, 
#             )

In [ ]:
!du -sh json

In [ ]:
sitecode = '302_OR_SNTL'
df = pd.read_json(f'./json/{sitecode}.json')

In [ ]:
#df.melt('datetime') #6575 rows!

In [ ]:
import altair as alt

In [ ]:
# https://altair-viz.github.io/user_guide/data.html#including-index-data

#data = df
data = f'https://raw.githubusercontent.com/scottyhq/snotel/main/json/{sitecode}.json'
#data = './json/302_OR_SNTL.json' # doesn't plot in jupyernotebook, maybe on webste though?

chart = alt.Chart(data, title=sitecode).transform_fold(
    ['SNWD', 'WTEQ'],
    as_=['variable', 'depth, swe (m)'],
).mark_line().encode(
    x='datetime:T',
    y='depth, swe (m):Q',
    color='variable:N',
    tooltip=['datetime:T','SNWD:Q','WTEQ:Q'],
).properties(width=600)

chart.interactive()
chart

In [ ]:
chart.save('302_OR_SNTL.html')

In [ ]:
# NOTE {'$schema': 'https://vega.github.io/schema/vega-lite/v4.17.0.json',
# Determines what javascript libraries should be when folium writes out HTML
# {'$schema': 'https://vega.github.io/schema/vega-lite/v4.17.0.json',
# https://github.com/python-visualization/folium/pull/1525

from pprint import pprint
my_chart = chart.to_dict()
pprint(my_chart)

In [ ]:
# Add MGRS square to each snotel station
#!pip install mgrs
import mgrs

m = mgrs.MGRS()

def get_mgrs_square(longitude, latitude):
    c = m.toMGRS(latitude, longitude, MGRSPrecision=0)
    return c

In [ ]:
get_mgrs(gf.longitude[0], gf.latitude[0])

In [ ]:
#gf['mgrs'] = 
gf['mgrs'] = gf.apply(lambda x: get_mgrs(x.longitude, x.latitude), axis=1)
gf.head()

In [ ]:
gf.to_file('snotel-sites.geojson', driver='GeoJSON')

In [ ]:
import geopandas as gpd
gf = gpd.read_file('snotel-sites.geojson')
gf.head()

In [ ]:
import os
def has_timeseries(code):
    return os.path.isfile(f'parquet/{code}.parquet')

has_timeseries(gf.code[0])
gf['timeseries'] = gf.apply(lambda x: has_timeseries(x.code), axis=1)
gf.head()

In [ ]:
gf[gf.code == '302_OR_SNTL']

In [ ]:
for i,row in gf[:3].iterrows():
    print(row.code)

In [ ]:
len(gf[gf.timeseries])

In [ ]:
# NOTE: it seems this is convenient for a dataframe, but if you want custom popups for each one
# need to iterate (see next cell)

import folium
import json

m = folium.Map(
    location=(47.6062, -122.3321),
    tiles="stamenterrain",
    zoom_start= 4,
    control_scale = True,
)

# Can also use geopandas explore() method here
mymarker = folium.Marker(icon=folium.Icon(color='green')) # Having trouble overriding styling...
#mystyle = lambda x: {'color': 'cyan' if x['properties']['timeseries']==True else 'magenta'}
mymarker = folium.Circle(fill=False, radius=200)
#snotel_stations = "https://raw.githubusercontent.com/scottyhq/snotel/main/snotel-sites.geojson"
#snotel_stations = gf #Timestamp not serialiable
#snotel_stations = 'file://snotel-sites.geojson' # does not show data from local file...
#snotel_stations = json.load(open("snotel-sites.geojson")) #works in JNotebook
snotel_stations = './snotel-sites.geojson' #works!
gjson = folium.GeoJson(snotel_stations, 
                       embed=False, 
                       marker=mymarker,
                       style_function=mystyle, 
                       name="snotel").add_to(m)

folium.features.GeoJsonTooltip(fields=["code"], labels=False).add_to(gjson)
folium.features.GeoJsonPopup(fields=all_fields, labels=True).add_to(gjson)

# Add popup plot with timeseries
# my_chart['data']['url'] = f'https://raw.githubusercontent.com/scottyhq/snotel/main/json/{code}.json'
# folium.Marker(
#     location = [45.21328,-117.192581],
#     # NOTE: chart width is 600, so this adjustment makes it fit nicely
#     popup = folium.Popup(min_width=700, max_width=700).add_child(folium.VegaLite(my_chart)),
# ).add_to(m)


# NOTE: is there a way to do this a function / variable map to make HTML more concise?
plots_group = folium.FeatureGroup(show=False, name='timeseries')
gfp = gf[gf.timeseries] # only stations with timesries data
for i,row in gfp.iterrows():
    my_chart['data']['url'] = f'https://raw.githubusercontent.com/scottyhq/snotel/main/json/{row.code}.json'
    folium.Marker(
         lazy = True,
         location = [row.latitude,row.longitude],
         #NOTE: chart width is 600, so this adjustment makes it fit nicely
         popup = folium.Popup(min_width=700, max_width=700).add_child(folium.VegaLite(my_chart)),
    ).add_to(plots_group) 

plots_group.add_to(m)
        
folium.LayerControl().add_to(m)

m

In [ ]:
m.save('index.html')

In [ ]:
# It is unclear to me how to easily add custom VEGA-Lite plots into GeoJsonpopups...
# https://github.com/python-visualization/folium/issues/1362
# https://github.com/python-visualization/folium/issues/1315#issuecomment-621016017

# 1. Make sure to lazy load
# 2. Easy syntax to create
# 3. Efficient HTML / javascript (don't want tons of entries...)


# Seems slow to load all stations even with lazy=True...
# Apparently not lazy....